## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [5]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Weather Description
0,0,Rikitea,-23.1203,-134.9692,78.22,74,100,17.02,light rain
1,1,Hermanus,-34.4187,19.2345,60.91,72,93,9.89,overcast clouds
2,2,Roma,43.2128,-75.4557,28.96,92,100,4.61,fog
3,3,Nicoya,10.1483,-85.4520,74.75,74,3,2.51,clear sky
4,4,Severo-Kurilsk,50.6789,156.1250,30.94,88,100,10.27,light snow


In [6]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip?  "))
max_temp = float(input("What is the maximum temperature you would like for your trip?  "))

What is the minimum temperature you would like for your trip?  62
what is the maximum temperature you would like for your trip?  85


In [7]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                      (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(5)

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Weather Description
0,0,Rikitea,-23.1203,-134.9692,78.22,74,100,17.02,light rain
3,3,Nicoya,10.1483,-85.4520,74.75,74,3,2.51,clear sky
6,6,Hithadhoo,-0.6000,73.0833,81.73,74,100,10.11,overcast clouds
7,7,Port Alfred,-33.5906,26.8910,69.84,75,98,15.48,overcast clouds
14,14,Muli,2.9167,73.5667,81.63,69,1,9.98,clear sky


In [9]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

Unnamed: 0             286
City                   286
Lat                    286
Lng                    286
Max Temp               286
Humidity               286
Cloudiness             286
Wind Speed             286
Weather Description    286
dtype: int64

In [10]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.


In [14]:
# 5a. Create DataFrame called hotel_df to store hotel names along with weather info
hotel_df = preferred_cities_df[["City", "Max Temp", "Weather Description", "Humidity", "Cloudiness", "Wind Speed", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Max Temp,Weather Description,Humidity,Cloudiness,Wind Speed,Lat,Lng,Hotel Name
0,Rikitea,78.22,light rain,74,100,17.02,-23.1203,-134.9692,
3,Nicoya,74.75,clear sky,74,3,2.51,10.1483,-85.4520,
6,Hithadhoo,81.73,overcast clouds,74,100,10.11,-0.6000,73.0833,
7,Port Alfred,69.84,overcast clouds,75,98,15.48,-33.5906,26.8910,
14,Muli,81.63,clear sky,69,1,9.98,2.9167,73.5667,
15,Mbanza-Ngungu,65.46,light rain,98,100,3.20,-5.2500,14.8667,
18,Codrington,68.00,broken clouds,59,71,16.15,-38.2667,141.9667,
24,Castro,64.17,moderate rain,98,100,4.05,-24.7911,-50.0119,
25,Esteio,83.05,few clouds,79,20,3.00,-29.8614,-51.1792,
28,Avarua,82.45,overcast clouds,78,97,17.27,-21.2078,-159.7750,


In [15]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Add the latitude and longitude to location key for the params dictionary
    params["location"] = f"{lat}, {lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params = params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [20]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df = hotel_df.dropna()

In [22]:
# 8a. Create the output File (CSV)

# 8b. Export the City_Data into a csv
hotel_df.to_csv("Vacation_Search/Vacation_Search.csv", index_label="City_ID")

In [24]:
# 9. Using the template add city name, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>City Name</dt><dd>{City}</dd>
<dt>Weather Description</dt><dd>{Weather Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp}  F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [25]:
# 11a. Add a marker layer for each city to the map. 
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=2.0)
heat_layer = gmaps.heatmap_layer(locations, weights = max_temp, dissipating = False,
                                max_intensity = 300, point_radius = 4)
marker_layer = gmaps.marker_layer(locations, info_box_content = hotel_info)

fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))